## 🥈 Camada Silver – Validar e Tratar


####🧪 Camada Silver – Tratamento e Enriquecimento

In [0]:
# Importação
from pyspark.sql.functions import trim, col, when, lit
from pyspark.sql.types import StringType

# Leitura das tabelas bronze
df_transacoes = spark.table("adb_cliente_savana_prd.daniel_bronze.transacoes")
df_clientes = spark.table("adb_cliente_savana_prd.daniel_bronze.clientes")

# Join entre as tabelas de transações e clientes, mantendo apenas as transações que possuem data não nula, Remove duplicatas para garantir que cada transação seja única.
df_silver = df_transacoes.join(
    df_clientes,
    col("transacoes.ID_Transacao") == col("clientes.Transacao"),
    "inner"
).filter(
    col("clientes.Dt_Transacao").isNotNull()
).distinct()

# Limpeza de espaços extras
for campo in ["Nome", "Cidade", "Meio_de_Pgmt"]:
    if campo in df_silver.columns:
        df_silver = df_silver.withColumn(campo, trim(col(campo)))

# Correções manuais de erros de Nome conhecidos
df_silver = df_silver.withColumn("Nome",
    when(col("Nome") == "Ana", "Ana Costa")
    .when(col("Nome") == "Carlos", "Carlos Pereira")
    .when(col("Nome") == "Fernanda", "Fernanda Gomes")
    .when(col("Nome") == "Gabriel", "Gabriel Martins")
    .when(col("Nome") == "Jos", "José da Silva")
    .when(col("Nome") == "Jos da Silva", "José da Silva")
    .when(col("Nome") == "Maria", "Maria Oliveira")
    .when(col("Nome") == "Mariana", "Mariana Almeida")
    .when(col("Nome") == "Ricardo", "Ricardo Lima")
    .when(col("Nome") == "Roberto", "Roberto Santos")
    .when(col("Nome") == "Tatiane", "Tatiane Ramos")
    .otherwise(col("Nome"))
)

df_silver = df_silver.withColumn("Cidade",
    when(col("Cidade") == "So Paulo", "São Paulo")
    .when(col("Cidade") == "Bralia", "Brasília")
    .when(col("Cidade") == "Braslia", "Brasília")
    .when(col("Cidade") == "Goinia", "Goiânia")
    .when(col("Cidade") == "Joo Pessoa", "João Pessoa")
    .otherwise(col("Cidade"))
)

def padronizar_nulls(df):
    for campo in df.schema.names:
        if df.schema[campo].dataType == StringType():
            df = df.withColumn(campo, when(trim(col(campo)) == "", None).otherwise(col(campo)))
    return df

# Qualquer Dado que estiver Duplicado ele será Eliminado
df_drop_duplicado = df_silver.dropDuplicates([
    "ID", "Nome", "Transacao", "ID_Transacao", "dt_transacao", "Cidade"
])

# Aplicar ao df_silver
df_silver = padronizar_nulls(df_silver)

# Exibir
display(df_silver)

In [0]:
# Conta total de registros
total_registros = df_silver.count()

# Conta total de IDs distintos
ids_unicos = df_silver.select("ID").distinct().count()

# Validação
if total_registros == ids_unicos:
    print("✅ Todos os registros da coluna 'ID' são diferentes (únicos).")
else:
    print(f"⚠️ Existem IDs duplicados. Total de registros: {total_registros}, IDs únicos: {ids_unicos}")


####🧪 Camada Silver – Quebra de Duplicados

In [0]:
# Qualquer Dado que estiver Duplicado ele será Eliminado
df_silver_finalizado = df_silver.dropDuplicates([
    "ID", "Nome", "Transacao", "ID_Transacao", "dt_transacao", "Cidade"
])
display(df_silver_finalizado)

In [0]:
# Conta total de registros
total_registros = df_silver_finalizado.count()

# Conta total de IDs distintos
ids_unicos = df_silver_finalizado.select("ID").distinct().count()

# Validação
if total_registros == ids_unicos:
    print(" Todos os registros da coluna 'ID' são diferentes (únicos).")
else:
    print(f" Existem IDs duplicados. Total de registros: {total_registros}, IDs únicos: {ids_unicos}")

####🧪 Camada Silver – Subindo a Tabela

In [0]:
# (Opcional) Remover a Tabela antiga visando Evitar Conflitos
spark.sql("DROP TABLE IF EXISTS adb_cliente_savana_prd.daniel_silver.BaseCliente_Transacoes")

# Salvar na camada Silver
df_silver_finalizado.write.format("delta").mode("overwrite") \
        .option("mergeSchema", "true").saveAsTable("adb_cliente_savana_prd.daniel_silver.BaseClientes_Transacoes")


### 🚨 Alerta

###### Observações: A coluna ID_Cliente aqui representa o cliente que fez a transação. Os IDs dos clientes vão até pelo menos 389, enquanto na tabela de clientes o maior ID parece ser bem menor (em torno de 5 nas primeiras linhas).

###❗ Problema de Consistência:
###### Não é possível fazer a junção diretamente, pois: A tabela de transações possui ID_Cliente com valores que não existem na tabela de clientes. O campo Transacao da tabela de clientes parece representar o número da transação, mas não há ligação clara com ID_Transacao.

### ✅ Recomendações:
###### Verifique a origem das tabelas: parece que elas vieram de fontes diferentes ou uma delas está desatualizada/incompleta. Corrija a tabela de clientes ou reavalie a tabela de transações. A única forma viável de fazer a junção é se: O campo ID da tabela de clientes bater com o ID_Cliente da tabela de transações.

### 🏁 Decisão:
###### O projeto é considerado um Prototipo com fins academicos, foi criado esse Markdown justamente para alertar sobre isso, Algumas decições aplicadas aqui seriam desconsideradas em um Projeto Real

### ℹ️ Fontes:
###### Para chegar as devidas conclusões foi usado algumas rodas de conversar com o Time sobre eticas de um Engenheiro de Dados, Validação junto a uma Inteligencia Artificial (ChatGPT)

### 💼 Projeto Alternativo
###### O Projeto alternativo ultiliza de uma Base de Dados mais coesa e não sofre com esse Tipo de Problema da Versão do Desafio